## Importing Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split  # Import train_test_split function

## Generating Synthetic Data 

In [2]:
# Define time periods (one day)
start_time = datetime(2023, 9, 23, 0, 0, 0)
end_time = datetime(2023, 9, 23, 23, 59, 59)
time_step = timedelta(minutes=15)

# Generate timestamps
timestamps = [start_time + i * time_step for i in range(int((end_time - start_time) / time_step))]

# Generate synthetic data for temperature, lighting intensity, occupancy, and energy consumption
num_samples = len(timestamps)
temperature = np.random.uniform(18, 28, num_samples)  # Temperature in Celsius
lighting_intensity = np.random.uniform(0, 1000, num_samples)  # Lighting intensity in lux
occupancy = np.random.choice([0, 1], num_samples, p=[0.7, 0.3])  # 0 - No occupancy, 1 - Occupied

# Calculate energy consumption based on temperature, lighting, and occupancy (simplified model)
energy_consumption = (temperature * 2 + lighting_intensity * 0.01 + occupancy * 10 +
                      np.random.normal(0, 5, num_samples))

# Create a DataFrame
data = pd.DataFrame({'Timestamp': timestamps, 'Temperature': temperature,
                     'Lighting_Intensity': lighting_intensity, 'Occupancy': occupancy,
                     'Energy_Consumption': energy_consumption})

# Save the synthetic data to a CSV file
data.to_csv('energy_data.csv', index=False)


## Preprocessing the Generated Data

In [3]:
# Loading synthetic data from the CSV file
data = pd.read_csv('energy_data.csv')

# Converting the 'Timestamp' column to datetime
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

# Extracting time-of-day and day-of-week features from 'Timestamp' column
data['Hour'] = data['Timestamp'].apply(lambda x: x.hour)
data['DayOfWeek'] = data['Timestamp'].apply(lambda x: x.weekday())

# Drop the original 'Timestamp' column as it's no longer needed
data.drop(columns=['Timestamp'], inplace=True)



In [4]:
data.head()

,Temperature,Lighting_Intensity,Occupancy,Energy_Consumption,Hour,DayOfWeek
0,22.728262,176.658757,1,59.817487,0,5
1,20.393047,639.113223,0,55.153046,0,5
2,22.223302,169.765727,0,43.321596,0,5
3,26.036642,977.149702,1,69.798575,0,5
4,24.867193,131.744187,0,41.052564,1,5


In [5]:
data.tail()

,Temperature,Lighting_Intensity,Occupancy,Energy_Consumption,Hour,DayOfWeek
90,25.648011,681.114458,0,64.134331,22,5
91,27.751920,0.245854,0,60.139873,22,5
92,21.031368,942.709911,1,69.589495,23,5
93,20.466279,22.545846,0,36.320134,23,5
94,18.586567,6.376175,0,32.195033,23,5


In [6]:
data.shape

(95, 6)

In [7]:
data.describe()

,Temperature,Lighting_Intensity,Occupancy,Energy_Consumption,Hour,DayOfWeek
count,95.000000,95.000000,95.000000,95.000000,95.000000,95.0
mean,23.151428,522.707044,0.242105,53.740555,11.378947,5.0
std,2.697454,276.418798,0.430630,9.167954,6.893072,0.0
min,18.040658,0.245854,0.000000,32.195033,0.000000,5.0
25%,21.224298,294.392777,0.000000,47.287301,5.500000,5.0
50%,23.420130,569.677009,0.000000,53.888769,11.000000,5.0
75%,25.251906,718.519990,0.000000,60.028278,17.000000,5.0
max,27.838620,983.855304,1.000000,76.305875,23.000000,5.0


## Splitting The Data into Test and Train Data

In [8]:
# Split the data into training and testing sets
X = data.drop(columns=['Energy_Consumption'])
y = data['Energy_Consumption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scaling The Data using Standardization (Z-score Scaling)

In [9]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Apply standardization to the features you want to normalize (e.g., 'Temperature' and 'Lighting_Intensity')
data[['Temperature', 'Lighting_Intensity']] = scaler.fit_transform(data[['Temperature', 'Lighting_Intensity']])

## Building the Optimization Model

#### Using a Random Forest Regressor

In [10]:
# Initialize and train the Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 35.21166188692909


#### Using Gradient Boosting

In [11]:
# Initialize and train the Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Make predictions
y_gb_pred = gb_model.predict(X_test)

# Evaluate the Gradient Boosting model
mse_gb = mean_squared_error(y_test, y_gb_pred)
print(f'Gradient Boosting Mean Squared Error: {mse_gb}')

Gradient Boosting Mean Squared Error: 36.78880685993483


#### Finetuning Gradient Boosting Model

In [12]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 3, 4]
}

grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_gb_model = grid_search.best_estimator_
y_best_gb_pred = best_gb_model.predict(X_test)
mse_best_gb = mean_squared_error(y_test, y_best_gb_pred)
print(f'Best Gradient Boosting Model Mean Squared Error: {mse_best_gb}')

Best Gradient Boosting Model Mean Squared Error: 37.21089443987587


#### Using Neural Network

In [13]:
# Standardize the input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple Neural Network model
nn_model = keras.Sequential([
    keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer
])

# Compile the model
nn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the Neural Network
nn_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=0)

# Make predictions
y_nn_pred = nn_model.predict(X_test_scaled)

# Evaluate the Neural Network model
mse_nn = mean_squared_error(y_test, y_nn_pred)
print(f'Neural Network Mean Squared Error: {mse_nn}')

1/1 [==============================] - 0s 154ms/step
Neural Network Mean Squared Error: 901.9161091654194


#### Using Linear Regression

In [14]:
from sklearn.linear_model import LinearRegression

# Initialize and train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
y_lr_pred = lr_model.predict(X_test)

# Evaluate the Linear Regression model
mse_lr = mean_squared_error(y_test, y_lr_pred)
print(f'Linear Regression Mean Squared Error: {mse_lr}')

Linear Regression Mean Squared Error: 25.828856801713016


#### 

#### 

#### 